Testing Notebook 

0. Set up of notebook

In [9]:
%cd ..
%load_ext autoreload
%autoreload 2

c:\Users\benjo\OneDrive\Documents
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


1. data_loader.py testing

In [10]:
from quant_tools.data_loader import get_data
df = get_data("AAPL", "2020-01-01", "2020-06-01")
print(df.head())

Price              Adj Close      Close       High        Low       Open  \
Ticker                  AAPL       AAPL       AAPL       AAPL       AAPL   
Date       Ticker                                                          
2020-01-02 AAPL    72.620842  75.087502  75.150002  73.797501  74.059998   
2020-01-03 AAPL    71.914818  74.357498  75.144997  74.125000  74.287498   
2020-01-06 AAPL    72.487854  74.949997  74.989998  73.187500  73.447502   
2020-01-07 AAPL    72.146935  74.597504  75.224998  74.370003  74.959999   
2020-01-08 AAPL    73.307503  75.797501  76.110001  74.290001  74.290001   

Price                 Volume  
Ticker                  AAPL  
Date       Ticker             
2020-01-02 AAPL    135480400  
2020-01-03 AAPL    146322800  
2020-01-06 AAPL    118387200  
2020-01-07 AAPL    108872000  
2020-01-08 AAPL    132079200  


2. Analysis.py Testing

In [11]:
import quant_tools.analysis as qa
import pandas as pd

close = prices = pd.Series(
    [100, 110, 105, 120],
    index=pd.date_range("2020-01-01", periods=4)
)

log_rets = qa.calculate_log_returns(close)
simp_rets = qa.calculate_returns(close)

ema20 = qa.calculate_moving_avg(close, 20)

vol = qa.calculate_volatility(log_rets)
sharpe = qa.calculate_sharpe_ratio(log_rets, risk_free=0.02)
mdd = qa.calculate_drawdown(close)

print(log_rets,simp_rets,ema20,vol,sharpe,mdd)

2020-01-02    0.095310
2020-01-03   -0.046520
2020-01-04    0.133531
Freq: D, dtype: float64 2020-01-02    0.100000
2020-01-03   -0.045455
2020-01-04    0.142857
Freq: D, dtype: float64 2020-01-01    100.000000
2020-01-02    100.952381
2020-01-03    101.337868
2020-01-04    103.115214
Freq: D, dtype: float64 1.5059207657399618 10.156584010697738 -0.045454545454545414


In [12]:
simp = pd.Series([0.1, -0.05, 0.142857], index=pd.date_range("2020-01-02", periods=3))

print("CumRet:", qa.cum_returns(simp))                
print("CAGR:", qa.cagr(simp, periods_per_year=3))            

print("Sortino:", qa.sortino_ratio(simp))                    
print("Calmar:", qa.calmar_ratio(simp, periods_per_year=3))  

print("VaR 5% hist:", qa.VaR_calc(simp, 0.05))          
print("ES 5% hist:", qa.expected_shortfall(simp, 0.05))      

CumRet: 0.19428556499999994
CAGR: 0.19428556499999994
Sortino: 20.41006642391935
Calmar: 3.8857112999999868
VaR 5% hist: -0.035
ES 5% hist: -0.05
